In [22]:
#Getting the data
import pandas as pd
from sets import Set

df = pd.read_csv("srep00196-s2.csv",header=None)
unique_ingredients = [i for i in df[0].unique()]
unique_ingredients = set(unique_ingredients + [i for i in df[1].unique()])

with open("Ingredients.csv","w") as f:
    for i in unique_ingredients:
        f.write(i + '\n')

#print unique_ingredients
#print unique_ingredients.pop()
#unique_ingredients = 